### Initialisation

In [ ]:
!pip install flair
!pip install yellowbrick
!pip install keras
!pip install sentence-transformers
!pip install keras
!pip install tensorflow

     |████████████████████████████████| 286kB 6.7MB/s 
     |████████████████████████████████| 798kB 9.2MB/s 
     |████████████████████████████████| 776.8MB 20kB/s 
     |████████████████████████████████| 2.3MB 31.0MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 71kB 7.0MB/s 
     |████████████████████████████████| 1.2MB 27.2MB/s 
     |████████████████████████████████| 19.7MB 469kB/s 
     |████████████████████████████████| 983kB 30.3MB/s 
     |████████████████████████████████| 61kB 7.1MB/s 
     |████████████████████████████████| 901kB 40.5MB/s 
     |████████████████████████████████| 3.3MB 17.1MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-cp37-none-any.whl size=9705 sha256=2ba81a8767cc03f83fd3de180c58078ba42b04fbe44a23093f13077b2224b74e
  Stored in directory: /root/.cache/pip/wheels/81/d0/d7/d9983facc6f2775411803e0e2d30ebf98efbf2fc6e57701e09
Successf

In [ ]:
!pip install yellowbrick

In [ ]:

from sklearn.model_selection import cross_validate as cross_validation, ShuffleSplit, cross_val_score, train_test_split, KFold, StratifiedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score, auc
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
import pandas as pd
import numpy as np
import time
from scipy.sparse import csr_matrix, hstack
from gensim.models import KeyedVectors
from gensim.models.wrappers import FastText
from flair.embeddings import FlairEmbeddings, TransformerWordEmbeddings , TransformerDocumentEmbeddings, SentenceTransformerDocumentEmbeddings
from flair.data import Sentence
import io
from google.colab import files
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import *
from sklearn.ensemble import *
from sklearn.neighbors import *
from sklearn.neural_network import *
from sklearn.linear_model import *
from keras.models import Model
from sklearn.naive_bayes import *
from gensim.models.wrappers import FastText 
import gensim
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from sklearn.linear_model import RidgeClassifierCV
import matplotlib.pyplot as plt
import pickle


In [ ]:
uploaded = files.upload()

Saving test_tweets.pkl to test_tweets.pkl
Saving train_tweets.pkl to train_tweets.pkl


In [ ]:
df_train = pd.read_pickle(io.BytesIO(uploaded['train_tweets.pkl']))
df_test = pd.read_pickle(io.BytesIO(uploaded['test_tweets.pkl']))

In [ ]:
le = LabelEncoder()
target_labels_train = df_train['Target']
target_labels_test = df_test['Target']
stance_labels_train = df_train['Stance']
stance_labels_test =   df_test['Stance']
opinion_labels_train = df_train['Opinion Towards']
opinion_labels_test = df_test['Opinion Towards']


def one_hot_encoding(label):
        label_le= le.fit_transform(label)
        return label_le

target_labels_train = one_hot_encoding(target_labels_train)
target_labels_test = one_hot_encoding(target_labels_test)
stance_labels_train = one_hot_encoding(stance_labels_train)
stance_labels_test = one_hot_encoding(stance_labels_test)
opinion_labels_train = one_hot_encoding(opinion_labels_train)
opinion_labels_test  = one_hot_encoding(opinion_labels_test)

In [ ]:
classifiers = []
classifiers.append(SVC(kernel='rbf'))
classifiers.append(LinearSVC())
classifiers.append(MLPClassifier())
classifiers.append(LogisticRegression(solver='liblinear'))

In [ ]:
folds = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 0)
to_join = lambda x : " ".join(x)
df_train['train'] = df_train.tokenize.apply(to_join)
df_test['test'] = df_test.tokenize.apply(to_join)
Train_X, Test_X, Train_Y, Test_Y = df_train['train'], df_test['test'], opinion_labels_train, opinion_labels_test 
Train_Y = np.array(Train_Y)
Test_Y = np.array(Test_Y)




## Text Representation Vectors


In [ ]:
# # get pretrained models
GLOVE_DIM = 200
FASTTEXT_DIM = 300
WORD2VEC_DIM = 300




In [ ]:
bert_emb = SentenceTransformerDocumentEmbeddings('roberta-base')
model_w2v = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/GoogleNews-vectors-negative300.bin', binary=True)
model_glove =pd.read_pickle('/content/drive/MyDrive/glove_dict.pickle')

Exception when trying to download http://sbert.net/models/roberta-base.zip. Response 404
SentenceTransformer-Model http://sbert.net/models/roberta-base.zip not found. Try to create it from scratch
Try to create Transformer Model roberta-base with mean pooling


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Word embeddings with tf-idf scaling

In [ ]:
# function to get tfidf * word emb vector 
def tfidf_scaled_wv(data,model,dim):
    text = []
    for i in data['tokenize']:
        string = ' '.join(i)
        text.append(string)
    tfidf = TfidfVectorizer(stop_words =None)
    tf_idf_tr = tfidf.fit_transform(text)
    feature_names = tfidf.get_feature_names()
    sentences = data['tokenize'].values
    data_dtm = pd.DataFrame(tf_idf_tr.toarray(), columns = feature_names)
    tfidf_sent_vectors = [] 
    row = 0
    for sentence in sentences:
        sent_vec = np.zeros(dim)
        weight_sum = 0
        for word in sentence:
            try:
                vec = model[word]
                tfidf = data_dtm.iloc[row][word]
                sent_vec += (vec * tfidf)
                weight_sum += tfidf
            except:
                pass
            # check if weight_sum is zero meaning all words in that tweet gave no context 
        if weight_sum != 0:     
            sent_vec  /= weight_sum     
        tfidf_sent_vectors.append(sent_vec)
        row+=1
    return tfidf_sent_vectors
    
    

### Sentence embedding BERT 

In [ ]:
def get_embeddings(text):
    sentence = Sentence(text)
    bert_emb.embed(sentence)
    return sentence.embedding




def sentence_vec(data):
    # the returned embedding is a tensor so we used the pytorch method to detach into a numpy array
    detach = lambda x : x.detach().numpy()
    emb  = lambda x : get_embeddings(x)
    data['sen_emb'] = data.Processed_Tweet.apply(emb)
    data['sen_emb'] = data.sen_emb.apply(detach)
#     data['sen_emb'] = data.sen_emb.apply(lambda x: np.array(x))

In [ ]:
def apply_embeddings(df_train,df_test,model,dim):
   train_emb = tfidf_scaled_wv(df_train,model,dim)
   test_emb = tfidf_scaled_wv(df_test,model,dim)
   return np.array(train_emb), np.array(test_emb)


def apply_sentence_embeddings(df_train,df_test):
    sentence_vec(df_train)
    sentence_vec(df_test)
   
  

In [ ]:
def hashtag_vectorizer(df_train,df_test):
     dummy_train = df_train.Hashtag.apply(lambda x : ' '.join(x))   
     dummy_test = df_test.Hashtag.apply(lambda x : ' '.join(x)) 

     cv = CountVectorizer(binary=True,ngram_range=(1,1),analyzer ='word')
     cv_hashtag_train = cv.fit_transform(dummy_train)
     cv_hashtag_test = cv.transform(dummy_test)
     names = cv.get_feature_names()

     return cv_hashtag_train, cv_hashtag_test, names

In [ ]:
apply_sentence_embeddings(df_train,df_test)  

## Feature Engineering

In [ ]:
POS_ratio_train = df_train[['RatioADJ','RatioADP','RatioADV','RatioAUXCONJ',
                      'RatioDET','RatioINTJ','RatioNOUN','RatioNUM',
                      'RatioPART','RatioPRON','RatioPROPN','RatioPUNCT',
                      'RatioSCONJ','RatioSYM','RatioVERB','RatioX']].values

POS_ratio_test =  df_test[['RatioADJ','RatioADP','RatioADV','RatioAUXCONJ',
                      'RatioDET','RatioINTJ','RatioNOUN','RatioNUM',
                      'RatioPART','RatioPRON','RatioPROPN','RatioPUNCT',
                      'RatioSCONJ','RatioSYM','RatioVERB','RatioX']].values

In [ ]:
def unigram_features(df_train,df_test):
    cv = CountVectorizer(binary=True,ngram_range=(1,1),analyzer ='word')
    cv_train = cv.fit_transform(Train_X)
    cv_test = cv.transform(Test_X)
    return cv_train , cv_test

In [ ]:
def bigram_features(df_train,df_test):
    cv = CountVectorizer(binary=True,ngram_range=(1,2),analyzer ='word')
    cv_train = cv.fit_transform(Train_X)
    cv_test = cv.transform(Test_X)
    return cv_train , cv_test

In [ ]:
def trigram_features(df_train,df_test):
    cv = CountVectorizer(binary=True,ngram_range=(1,3),analyzer ='word')
    cv_train = cv.fit_transform(Train_X)
    cv_test = cv.transform(Test_X)
    names = cv.get_feature_names()
    return cv_train , cv_test,names

In [ ]:
#create vector made of of count vectorizer 1,3 n grams word with POS tags ratios 
# Hashtags unigram vectorizer 
def bag_of_words_POSratio_Hashtag(df_train,df_test):

    Ntrain_tags = df_train[['num#']].values
    Ntest_tags = df_test[['num#']].values
                                        
    cv_hashtag_train ,cv_hashtag_test = hashtag_vectorizer(df_train,df_test)

    cv_train, cv_test = trigram_features(df_train,df_test)

    combined_features_train = hstack([cv_train, cv_hashtag_train], 'csr')
    combined_features_train = hstack([combined_features_train,
                                     POS_ratio_train],'csr' )
   
   
    combined_features_test = hstack([cv_test, cv_hashtag_test], 'csr')
    combined_features_test = hstack([combined_features_test,
                                     POS_ratio_test],'csr' )
    

   
   
    return combined_features_train , combined_features_test


def BOW_Hashtag(df_train,df_test):
    cv_train, cv_test ,names = trigram_features(df_train,df_test)
    cv_hashtag_train ,cv_hashtag_test,names1 = hashtag_vectorizer(df_train,df_test)
    combined_features_train = hstack([cv_train, cv_hashtag_train], 'csr')
    combined_features_test = hstack([cv_test, cv_hashtag_test], 'csr')
    all_features = []
    for n in names:
      all_features.append(n)
    for n in names1:
      all_features.append(n)
    
    return combined_features_train , combined_features_test , all_features


   

In [ ]:
def word_embeddings_POSratio_Hashtag(df_train,df_test,emb_train,emb_test):

    Ntrain_tags = df_train[['num#']].values
    Ntest_tags = df_test[['num#']].values
                                        
    cv_hashtag_train ,cv_hashtag_test = hashtag_vectorizer(df_train,df_test)

  
    combined_features_train = hstack([emb_train, cv_hashtag_train], 'csr')
    combined_features_train = hstack([combined_features_train,
                                     POS_ratio_train],'csr' )
    

    combined_features_test = hstack([emb_test, cv_hashtag_test], 'csr')
    combined_features_test = hstack([combined_features_test,
                                     POS_ratio_test],'csr' )
    


    return combined_features_train , combined_features_test


   
def word_emb_Hashtag(df_train,df_test,emb_train,emb_test):
    cv_hashtag_train ,cv_hashtag_test = hashtag_vectorizer(df_train,df_test)
    combined_features_train = hstack([emb_train, cv_hashtag_train], 'csr')
    combined_features_test = hstack([emb_test, cv_hashtag_test], 'csr')
    return combined_features_train, combined_features_test
    

In [ ]:
def sentence_embeddings_POSratio_Hashtag(df_train,df_test,emb_train,emb_test):
   
     Ntrain_tags = df_train[['num#']].values
     Ntest_tags = df_test[['num#']].values
                                        
     cv_hashtag_train ,cv_hashtag_test = hashtag_vectorizer(df_train,df_test)

     combined_features_train = hstack([emb_train, cv_hashtag_train], 'csr')
     combined_features_train = hstack([combined_features_train,
                                     POS_ratio_train],'csr' )
     

    

     combined_features_test = hstack([emb_test, cv_hashtag_test], 'csr')
     combined_features_test = hstack([combined_features_test,
                                     POS_ratio_test],'csr' )

     return combined_features_train , combined_features_test

 

def bert_Hashtag(df_train,df_test,emb_train,emb_test):
    cv_hashtag_train ,cv_hashtag_test = hashtag_vectorizer(df_train,df_test)
    combined_features_train = hstack([emb_train, cv_hashtag_train], 'csr')
    combined_features_test = hstack([emb_test, cv_hashtag_test], 'csr')
    return combined_features_train, combined_features_test 


### Basic Classifiers Run

In [ ]:
def basic_classifier_tester(classifiers,X_train,Y_train,X_test,Y_test):
    report_scores = {}
    for classifier in (classifiers):
        classifier.fit(X_train, Y_train)
        predictions = classifier.predict(X_test)
        report = classification_report(Y_test, predictions,output_dict=True)
        report_scores[classifier.__class__.__name__] = report
    return report_scores



def evaluate_models_hashtag_vectorizer(df_train,df_test):
     train_x , valid_x = hashtag_vectorizer(df_train,df_test)
     scores = basic_classifier_tester(classifiers,train_x,Train_Y,valid_x,Test_Y)
     return scores


def evaluate_models_Feature_Eng_BOW(df_train,df_test):
      train_x , valid_x = bag_of_words_POSratio_Hashtag(df_train,df_test)
      scores = basic_classifier_tester(classifiers,train_x,Train_Y,valid_x,Test_Y)
      return scores


def evaluate_models_Feature_Eng_w2v(df_train,df_test):
     train_emb , valid_emb = apply_embeddings(df_train,df_test,model_w2v,300)
     train_x , valid_x = word_embeddings_POSratio_Hashtag(df_train,df_test,train_emb,valid_emb)
     scores = basic_classifier_tester(classifiers,train_x,Train_Y,valid_x,Test_Y) 
     return scores


def evaluate_models_Feature_Eng_glove(df_train,df_test):
     train_emb , valid_emb = apply_embeddings(df_train,df_test,model_glove,200)
     train_x , valid_x = word_embeddings_POSratio_Hashtag(df_train,df_test,train_emb,valid_emb)
     scores = basic_classifier_tester(classifiers,train_x,Train_Y,valid_x,Test_Y)
     return scores


def evaluate_models_Feature_Eng_fasttext():
     train_emb , valid_emb = apply_embeddings(df_train,df_test,model_fasttext)
     train_x , valid_x = word_embeddings_POSratio_Hashtag(df_train,df_test,train_emb,valid_emb)
     scores = basic_classifier_tester(classifiers,train_x,train_Y,valid_x,test_Y) 
     return scores
    

def evaluate_models_Feature_Eng_BERT(df_train,df_test):
  train_emb  =  np.array(df_train['sen_emb'].tolist())
  valid_emb = np.array(df_test['sen_emb'].tolist())
  train_x , valid_x = sentence_embeddings_POSratio_Hashtag(df_train,df_test,train_emb,valid_emb)
  scores = basic_classifier_tester(classifiers,train_x,Train_Y,valid_x,Test_Y) 
  return scores


def evaluate_models_Feature_Eng_trigram(df_train,df_test):
   train_x , valid_x = trigram_features(df_train,df_test)
   scores = basic_classifier_tester(classifiers,train_x,Train_Y,valid_x,Test_Y) 
   return scores



def evaluate_BOW_Hashtag(df_train,df_test):
     train_x , valid_x = BOW_Hashtag(df_train,df_test)
     scores = basic_classifier_tester(classifiers,train_x,Train_Y,valid_x,Test_Y) 
     return scores

def evaluate_w2v_emb_Hashtag(df_train,df_test):
    train_emb , valid_emb = apply_embeddings(df_train,df_test,model_w2v,300)
    train_x , valid_x = word_emb_Hashtag(df_train,df_test,train_emb,valid_emb)
    scores = basic_classifier_tester(classifiers,train_x,Train_Y,valid_x,Test_Y) 
    return scores

def evaluate_bert_emb_Hashtag(df_train,df_test):
    train_emb  =  np.array(df_train['sen_emb'].tolist())
    valid_emb = np.array(df_test['sen_emb'].tolist())
    train_x, valid_x = bert_Hashtag(df_train,df_test,train_emb,valid_emb)
    scores = basic_classifier_tester(classifiers,train_x,Train_Y,valid_x,Test_Y) 
    return scores

def evaluate_glove_emb_Hashtag(df_train,df_test):
    train_emb , valid_emb = apply_embeddings(df_train,df_test,model_glove,200)
    train_x , valid_x = word_emb_Hashtag(df_train,df_test,train_emb,valid_emb)
    scores = basic_classifier_tester(classifiers,train_x,Train_Y,valid_x,Test_Y) 
    return scores



## Run All Models

In [ ]:
targets = ['hil','cc','ab','ath','fm','all']

def run_all_models(df_train,df_test,Train_Y,Test_Y):
  # first batch
    scores = []
    s = evaluate_models_Feature_Eng_trigram(df_train,df_test)
    scores.append(s)
    w2v_train , w2v_valid = apply_embeddings(df_train,df_test,model_w2v,300)
    s = basic_classifier_tester(classifiers,w2v_train,Train_Y,w2v_valid,Test_Y)
    scores.append(s)
    glove_train , glove_valid = apply_embeddings(df_train,df_test,model_glove,200)
    s = basic_classifier_tester(classifiers,glove_train,Train_Y,glove_valid,Test_Y)
    scores.append(s)
    train_emb  =  np.array(df_train['sen_emb'].tolist())
    valid_emb = np.array(df_test['sen_emb'].tolist())
    s = basic_classifier_tester(classifiers,train_emb,Train_Y,valid_emb,Test_Y)
    scores.append(s)
    # second batch
    s = evaluate_BOW_Hashtag(df_train,df_test)
    scores.append(s)
    s = evaluate_w2v_emb_Hashtag(df_train,df_test)
    scores.append(s)
    s = evaluate_glove_emb_Hashtag(df_train,df_test)
    scores.append(s)
    s = evaluate_bert_emb_Hashtag(df_train,df_test)
    scores.append(s)
    # third batch
    s = evaluate_models_Feature_Eng_BOW(df_train,df_test)
    scores.append(s)
    s = evaluate_models_Feature_Eng_w2v(df_train,df_test)
    scores.append(s)
    s = evaluate_models_Feature_Eng_glove(df_train,df_test)
    scores.append(s)
    s = evaluate_models_Feature_Eng_BERT(df_train,df_test)
    scores.append(s)
    return scores

def put_scores(s):
    scores = {}
    scores['ngram'] = s[0]
    scores['w2v'] = s[1]
    scores['glove'] = s[2]
    scores['BERT'] = s[3]
    scores['ngram_hashtag'] = s[4]
    scores['w2v_hashtag'] = s[5]
    scores['glove_hashtag'] = s[6]
    scores['bert_hashtag'] = s[7]
    scores['ngram_all'] = s[8]
    scores['w2v_all'] = s[9]
    scores['glove_all'] = s[10]
    scores['bert_all'] = s[11]
    return scores

def Evaluation():
   dicts = {}
   print('Evaluation beginning /////////////////////////')
   scores = run_all_models(df_train,df_test,Train_Y,Test_Y)
   scores  = put_scores(scores)
   dicts['All'] = scores
   print('Evaluation completed ////////////////////////')
   return dicts


In [ ]:
evaluation =Evaluation()

Evaluation beginning /////////////////////////


ValueError: ignored

In [ ]:
with open('evaluation_arguments.pickle', 'wb') as handle:
    pickle.dump(evaluation, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Error Analysis 

In [ ]:
def misclassification(clf,df_test):
   #index of misclassifications
   y_pred = clf.predict(Test_X)
   correct = y_pred == Test_Y
   df_test['Classified'] = correct
   classified = df_test[['Tweet','Classified','Target','Opinion Towards']]
   return classified 

In [ ]:
evaluation_argument = pd.read_pickle('/content/evaluation_arguments.pickle')

FileNotFoundError: ignored

In [ ]:
evaluation_argument['All']['ngram_hashtag']['LinearSVC']

In [ ]:
def finetune_linearsvc(Train_X,Train_Y,Test_X,Test_Y):
    C = [2**(-3),2**(-2),2**(-1),1,2,4,8,16,32,100,1000] 
    parameter_space = [ {'C': C}]
    clf_svm_linear = GridSearchCV(LinearSVC(), 
                               parameter_space,  
                               cv =folds,
                               n_jobs=-1,
                               verbose=True    
                              )    

    t0 =time.time()
# fit the model
    clf_svm_linear.fit(Train_X, Train_Y)
    t1= time.time()
    print("training time:", round(t1-t0, 3), "s") # the time would be round to 3 decimal in seconds
    print(clf_svm_linear.best_params_)

    svm = LinearSVC(C = clf_svm_linear.best_params_['C'],max_iter=1500)
    svm.fit(Train_X, Train_Y)
    predictions = svm.predict(Test_X)
    report = classification_report(Test_Y, predictions,output_dict=True)
    return predictions, svm
    

In [ ]:

# from sklearn.model_selection import learning_curve

# train, test , names = BOW_Hashtag(df_train,df_test)



# estimator =  LinearSVC()
# def plot_learning_curve(estimator, title, X, y, axes=None, ylim=None, cv=None,
#                         n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
#     """
#     Generate 3 plots: the test and training learning curve, the training
#     samples vs fit times curve, the fit times vs score curve.

#     Parameters
#     ----------
#     estimator : estimator instance
#         An estimator instance implementing `fit` and `predict` methods which
#         will be cloned for each validation.

#     title : str
#         Title for the chart.

#     X : array-like of shape (n_samples, n_features)
#         Training vector, where ``n_samples`` is the number of samples and
#         ``n_features`` is the number of features.

#     y : array-like of shape (n_samples) or (n_samples, n_features)
#         Target relative to ``X`` for classification or regression;
#         None for unsupervised learning.

#     axes : array-like of shape (3,), default=None
#         Axes to use for plotting the curves.

#     ylim : tuple of shape (2,), default=None
#         Defines minimum and maximum y-values plotted, e.g. (ymin, ymax).

#     cv : int, cross-validation generator or an iterable, default=None
#         Determines the cross-validation splitting strategy.
#         Possible inputs for cv are:

#           - None, to use the default 5-fold cross-validation,
#           - integer, to specify the number of folds.
#           - :term:`CV splitter`,
#           - An iterable yielding (train, test) splits as arrays of indices.

#         For integer/None inputs, if ``y`` is binary or multiclass,
#         :class:`StratifiedKFold` used. If the estimator is not a classifier
#         or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

#         Refer :ref:`User Guide <cross_validation>` for the various
#         cross-validators that can be used here.

#     n_jobs : int or None, default=None
#         Number of jobs to run in parallel.
#         ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
#         ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
#         for more details.

#     train_sizes : array-like of shape (n_ticks,)
#         Relative or absolute numbers of training examples that will be used to
#         generate the learning curve. If the ``dtype`` is float, it is regarded
#         as a fraction of the maximum size of the training set (that is
#         determined by the selected validation method), i.e. it has to be within
#         (0, 1]. Otherwise it is interpreted as absolute sizes of the training
#         sets. Note that for classification the number of samples usually have
#         to be big enough to contain at least one sample from each class.
#         (default: np.linspace(0.1, 1.0, 5))
#     """
#     if axes is None:
#         _, axes = plt.subplots(1, 3, figsize=(20, 5))

#     axes[0].set_title(title)
#     if ylim is not None:
#         axes[0].set_ylim(*ylim)
#     axes[0].set_xlabel("Training examples")
#     axes[0].set_ylabel("Score")

#     train_sizes, train_scores, test_scores, fit_times, _ = \
#         learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs,
#                        train_sizes=train_sizes,
#                        return_times=True)
#     train_scores_mean = np.mean(train_scores, axis=1)
#     train_scores_std = np.std(train_scores, axis=1)
#     test_scores_mean = np.mean(test_scores, axis=1)
#     test_scores_std = np.std(test_scores, axis=1)
#     fit_times_mean = np.mean(fit_times, axis=1)
#     fit_times_std = np.std(fit_times, axis=1)

#     # Plot learning curve
#     axes[0].grid()
#     axes[0].fill_between(train_sizes, train_scores_mean - train_scores_std,
#                          train_scores_mean + train_scores_std, alpha=0.1,
#                          color="r")
#     axes[0].fill_between(train_sizes, test_scores_mean - test_scores_std,
#                          test_scores_mean + test_scores_std, alpha=0.1,
#                          color="g")
#     axes[0].plot(train_sizes, train_scores_mean, 'o-', color="r",
#                  label="Training score")
#     axes[0].plot(train_sizes, test_scores_mean, 'o-', color="g",
#                  label="Cross-validation score")
#     axes[0].legend(loc="best")

#     # Plot n_samples vs fit_times
#     axes[1].grid()
#     axes[1].plot(train_sizes, fit_times_mean, 'o-')
#     axes[1].fill_between(train_sizes, fit_times_mean - fit_times_std,
#                          fit_times_mean + fit_times_std, alpha=0.1)
#     axes[1].set_xlabel("Training examples")
#     axes[1].set_ylabel("fit_times")
#     axes[1].set_title("Scalability of the model")

#     # Plot fit_time vs score
#     axes[2].grid()
#     axes[2].plot(fit_times_mean, test_scores_mean, 'o-')
#     axes[2].fill_between(fit_times_mean, test_scores_mean - test_scores_std,
#                          test_scores_mean + test_scores_std, alpha=0.1)
#     axes[2].set_xlabel("fit_times")
#     axes[2].set_ylabel("Score")
#     axes[2].set_title("Performance of the model")

#     return plt


# fig, axes = plt.subplots(3, 2, figsize=(10, 15))

# title = "Learning Curves (linear SVC)"
# # Cross validation with 100 iterations to get smoother mean test and train
# # score curves, each time with 20% data randomly selected as a validation set.
# cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)

# plot_learning_curve(estimator, title, train, Train_Y, axes=axes[:, 0], ylim=(0.7, 1.01),
#                     cv=cv, n_jobs=-1)

# plt.show()

In [ ]:
train, test , names = BOW_Hashtag(df_train,df_test)
folds = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 0)

predictions , svm = finetune_linearsvc(train,Train_Y,test,Test_Y)

In [ ]:
df_test['Opinion Towards'] = le.fit_transform(df_test['Opinion Towards'])

In [ ]:
df_test['predicted_arguments'] = predictions 

In [ ]:
mask = df_test['predicted_arguments'] == 0

In [ ]:
argumentative_tweets = df_test.loc[mask]

In [ ]:
argumentative_tweets.to_pickle('argumentative_tweets.pkl')

In [ ]:
train_, test_ = trigram_features(df_train,df_test)
folds = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 0)

report , svm_ = finetune_linearsvc(train_,Train_Y,test_,Test_Y)

In [ ]:
def misclassification(clf,df_test,valid_x):
   #index of misclassifications
   y_pred = clf.predict(valid_x)
   correct = y_pred == Test_Y
   df_test['Classified'] = correct
   classified = df_test[['Tweet','Classified','Target','Hashtag']]
   return classified

In [ ]:
classified = misclassification(svm,df_test,test)
classified_ = misclassification(svm_,df_test,test_)

In [ ]:
misc_ngram  = classified_.loc[classified_['Classified'] == False]
cc_hash = classified.loc[classified['Classified'] == True]

In [ ]:
corrected_tweets = []
corrected_hashtags = []
for index, row in cc_hash.iterrows():
    for index_,row_ in  misc_ngram.iterrows():
         if row['Tweet'] == row_['Tweet']:
               corrected_tweets.append(row['Tweet']) 
               corrected_hashtags.append(row['Hashtag'])        

In [ ]:
corrected_tweets

In [ ]:
from sklearn.feature_selection import SelectFromModel

In [ ]:
# sel = SelectFromModel(svm,prefit=True)
# sel.transform(train)
features = np.reshape(names, (-1, 1))

best_features = pd.DataFrame({ 'Rank': svm.coef_[0],
                               'vectors': names})

In [ ]:
def f_importances(coef, names):
    imp = coef[0]
    imp,names = zip(*sorted(zip(imp,names)))
    plt.barh(range(len(names)), imp, align='center')
    plt.yticks(range(len(names)), names)
    plt.show()


In [ ]:
best = best_features.sort_values(by='Rank',ascending=False)

In [ ]:
best.head(30)

In [ ]:
imp,names = zip(*sorted(zip(best['Rank'].iloc[:20],best['vectors'].iloc[:20])))
plt.figure(figsize=(5,6))
plt.barh(range(len(names)), imp, align='center')
plt.yticks(range(len(names)), names)

plt.title('The Top 20 features Measured by Linear SVC Coefficient Value')
plt.show()